<a href="https://colab.research.google.com/github/brendanlooker/colab-examples/blob/main/dataform/dataformAPI_CompilationResultsVariables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the Client Library

In [ ]:
!pip install google-cloud-dataform

In [ ]:

import logging
from google.cloud import dataform_v1beta1 as dataform

In [ ]:
# Authenticate via ADC (Application Default Credentials)

!gcloud auth application-default login

In [ ]:
# Complete set-up
gcp_project = "" # @param {type:"string"}

!gcloud auth application-default set-quota-project {gcp_project}

In [ ]:
# Initialise client

df_client = dataform.DataformClient()


In [ ]:
# Using Dataform's Compilation Overrides functionality allow for different Compilation Results to be created using different variable values
# Each new set of variable values requires a new Compilation to be created
# At executing time (i.e. when the pipline is run) the variable values used when creating the Compliation Result will be in scope

# https://cloud.google.com/dataform/docs/configure-compilation#about-api-compilation-overrides


# Here I am hard-coding a set of variable values, however it is also possible to retrieve from BQ (or any other datastore)

df_vars = {
    "var1": '4',
    "var2": '5',
    "var3": '6'
}

In [ ]:
# Use the Dataform API to Compile a workflow (i.e. Create a complilation result)

location = "europe-west1" # @param {type:"string"}
repo_name = "pipeline-orchestration-config" # @param {type:"string"}
bq_dataset = 'df_pipeline_orchestration' # @param {type:"string"}
branch = 'master' # @param {type:"string"}


repo_uri = f'projects/{gcp_project}/locations/{location}/repositories/{repo_name}'

request = dataform.CreateCompilationResultRequest(
    parent=repo_uri,
    compilation_result=dataform.types.CompilationResult(
        git_commitish=branch,
        code_compilation_config=dataform.types.CodeCompilationConfig(
            default_database=gcp_project,
            default_schema=bq_dataset,
            vars=df_vars
        )
    )
)

print(request)

response = df_client.create_compilation_result(request=request)
compilation_result = response.name
logging.info(f'compiled workflow {compilation_result}')
print(compilation_result)


In [ ]:
# Execute a workflow (Using the complilation result created in the previous step)


repo_uri = f'projects/{gcp_project}/locations/{location}/repositories/{repo_name}'



request = dataform.CreateWorkflowInvocationRequest(
    parent=repo_uri,
    workflow_invocation=dataform.types.WorkflowInvocation(
        compilation_result=compilation_result,
        invocation_config=dataform.InvocationConfig(
            included_tags=["user"],
        )
    )
)

print(request)

response = df_client.create_workflow_invocation(request=request)
workflow_invocation = response.name
logging.info(f'created workflow invocation {workflow_invocation}')
print (workflow_invocation)